In [1]:
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

# Quadratic Function Fitting using Least Squares

$$y=ax^2+bx+c$$

Find a, b, c  using LS by given 11 2D points, and plot it.

In [ ]:
PointsX = np.array([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5], dtype=np.float64)
PointsY = np.array([2.8, 1.0, -1.0, -2.3, -2.7, -3.0, -3.2, -2.0, -1.0, 0.4, 3.0], dtype=np.float64)
Weights = np.array([1, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1], dtype=np.float64)

# Multi-lines Fitting using Least Squares

$y=ax+b$

Find $a$, $b$ using LS: Find the intersections of red and green respectively.

## How to do?
1. Select some 2D points at pixel coordinates on the same line, and find the optimal line equation using LS
2. Repeat 1 for other lines in the same direction
3. Find the intersections of lines in (1)-(2) in the same direction
4. Repeat (1)-(3) for other directions(red/green)